# Lesson 4️⃣ `Web3.py` Simple Storage

In [33]:
import json
#!conda install jsonschema==3.2.0
from web3 import Web3
from solcx import compile_standard, install_solc
#install_solc("0.6.0")
import pprint
import os
from dotenv import load_dotenv


In [8]:
with open("./SimpleStorage.sol", "r") as file:
    simple_storage_file = file.read()
    #print(simple_storage_file)

In [10]:
# Solidity source code
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorage.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.6.0",
)

In [11]:
pp = pprint.PrettyPrinter(indent=1)
#pp.pprint(compiled_sol)

In [12]:
with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [13]:
# get bytecode
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
bytecode[:100]

'608060405234801561001057600080fd5b506105aa806100206000396000f3fe608060405234801561001057600080fd5b50'

In [14]:
# get abi
abi = json.loads(compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["metadata"])["output"]["abi"]
#abi

In [61]:
# w3 = Web3(Web3.HTTPProvider(os.getenv("RINKEBY_RPC_URL")))
# chain_id = 4
#
# For connecting to ganache
#w3 = Web3(Web3.HTTPProvider("http://0.0.0.0:8545"))
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
w3 = Web3(Web3.HTTPProvider("https://rinkeby.infura.io/v3/352df199ff044272a7fcb535e95351fd"))
#chain_id = 1337#5777
chain_id = 4 #<-Rinkeby

#my_address = "0xE6A9fa6AA4874812816b27AfF3c8d4FDa9Bb1fFf"
my_address = "0xffbD9E870E526F3e28C68A2f95923a17Fd70D52a" #<--my MetaMask

# Load private key from .env
load_dotenv()
private_key = os.getenv("PRIVATE_KEY")
private_key = os.getenv("METAMASKPRIVATE_KEY")

In [62]:
# Create the contract in Python
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)
print(SimpleStorage)

<class 'web3._utils.datatypes.Contract'>


In [63]:
# Get the latest transaction
nonce = w3.eth.getTransactionCount(my_address)
print(nonce)

3


In [64]:
# 1. Build transaction
# 2. Sign "
# 3. Send "

# Submit the transaction that deploys the contract
transaction = SimpleStorage.constructor().buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": nonce,
    }
)

In [65]:
print(transaction)

{'value': 0, 'gas': 366119, 'chainId': 4, 'gasPrice': 1000000024, 'from': '0xffbD9E870E526F3e28C68A2f95923a17Fd70D52a', 'nonce': 3, 'data': '0x608060405234801561001057600080fd5b506105aa806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100a85780638bab8dd51461016d5780639e7a13ad1461023c575b600080fd5b6100646102ea565b6040518082815260200191505060405180910390f35b6100a66004803603602081101561009057600080fd5b81019080803590602001909291905050506102f3565b005b61016b600480360360408110156100be57600080fd5b81019080803590602001906401000000008111156100db57600080fd5b8201836020820111156100ed57600080fd5b8035906020019184600183028401116401000000008311171561010f57600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290803590602001909291905050506102fd565b005b6102266004803603602081101561018357600080fd5b81019080803590602

In [66]:
# Sign the transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
print("Deploying Contract!")

Deploying Contract!


In [67]:
# Send it!
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

In [68]:
# Wait for the transaction to be mined, and get the transaction receipt
print("Waiting for transaction to finish...")
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Done! Contract deployed to {tx_receipt.contractAddress}")

Waiting for transaction to finish...
Done! Contract deployed to 0x26AD8d7740eD4595ea41557934ECb6d909297197


- `call` 👈🏻 simulates making a call, i.e. grabbing data from blockchain
- `transact` 👈🏻 simulates transactions in block chain, i.e. make a state change



In [69]:
# Working with deployed Contracts
simple_storage = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
print(f"Initial Stored Value {simple_storage.functions.retrieve().call()}") #<-call

greeting_transaction = simple_storage.functions.store(15).buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": nonce + 1,
    }
)

signed_greeting_txn = w3.eth.account.sign_transaction(
    greeting_transaction, private_key=private_key
)

tx_greeting_hash = w3.eth.send_raw_transaction(signed_greeting_txn.rawTransaction)
print("Updating stored Value...")

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_greeting_hash)
print(simple_storage.functions.retrieve().call())

Initial Stored Value 0
Updating stored Value...
15


## Results
On Etherscan (with my MetaMask Account) https://rinkeby.etherscan.io/address/0xffbD9E870E526F3e28C68A2f95923a17Fd70D52a:
[](screenshot_1.png)

--- 
# Installation ⚙️

### Install Ganache CLI
1. `node --version` #check node is installed
1. `npm install --global yarn`
2. `yarn --version` #check installation
4. `yarn global add ganache-cli`
5. `ganache-cli --version` #check installation

In [53]:
!ganache-cli

Ganache CLI v6.12.2 (ganache-core: 2.13.2)
Error: listen EADDRINUSE: address already in use 127.0.0.1:8545
    at Server.setupListenHandle [as _listen2] (net.js:1300:14)
    at listenInCluster (net.js:1348:12)
    at doListen (net.js:1487:7)
    at processTicksAndRejections (internal/process/task_queues.js:81:21)
